In [1]:
import utils
import pyproj
import numpy as np
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 999

from functools import partial
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from shapely.ops import transform
from shapely.geometry import Point, Polygon, MultiPolygon
from haversine import haversine, Unit

import warnings
warnings.filterwarnings("ignore")

# Process Restaurant Data

- Clean restaurant dataset
- Map each eatery to cuisine

In [2]:
# read restaurant data
restaurant_df = pd.read_csv("./data/restaurant_all.csv")

# convert categories to list
restaurant_df = utils.process_csv_lists(df=restaurant_df, columns=["categories"])

# remove restaurants with no lat long information
restaurant_df = restaurant_df[restaurant_df["lat"] != 0][restaurant_df["long"] != 0] 

# filter prices 
restaurant_df["price_per_pax"][restaurant_df["price_per_pax"] < 0] = 0

In [3]:
category_cuisine_mapping = {"Dim Sum": "Chinese", "Ramen": "Others", "Zi Char": "Chinese",
    "Pasta": "Western", "Korean Desserts": "Others", "Korean BBQ": "Others", "Korean Fried Chicken": "Others",
    "Mookata": "Others", "Teppanyaki": "Others", "Sushi": "Others", "Chirashi": "Others", "Chicken Rice": "Chinese",
    "Burgers": "Western", "Bak Kut Teh": "Chinese", "Char Kway Teow": "Chinese", "Steak": "Western",
    "Hot Pot": "Chinese", "Nasi Lemak": "Malay", "Pizza": "Western", "Argentinian": "Western", 
    "Brazilian": "Western",  "European": "Western", "Greek": "Western", "French": "Western",
    "Mediterranean": "Others", "Turkish": "Others", "Russian": "Western", "Taiwanese": "Chinese",
    "Indonesian": "Malay", "Spanish": "Mexican", "Salads": "Western", "Sandwiches": "Western", "Fast Food": "Western",
    "Cafes & Coffee": "Western", "Bubble Tea": "Chinese", "Breakfast & Brunch": "Western", "Bars": "Western",
    "Waffles": "Western", "Italian": "Western", "Middle Eastern": "Others", "Japanese": "Others",
    "Korean": "Others", "Thai": "Others", "Vietnamese": "Others", "Mexican": "Others", "Kopitiam": "Hawker Food",
    "Spanish": "Western", "Desserts": "Others", "Bread & Pastries": "Others", "Fruit Tea": "Chinese",
    "Halal": "Malay"
}

cuisines = ["Chinese", "Malay", "Indian", "Western", "Others", "Peranakan", "Hawker Food"]
for cat in cuisines:
    restaurant_df[cat] = [1 if cat in x else 0 for x in restaurant_df["categories"]]
for cat in category_cuisine_mapping.keys():
    cuisine = category_cuisine_mapping[cat]
    updated_row = restaurant_df[cuisine]
    for i, row in restaurant_df.iterrows():
        if cat in row["categories"]:
            updated_row[i] = 1
    restaurant_df[cuisine] = updated_row

restaurant_df = restaurant_df[["name", "term", "location", "lat", "long", "price_per_pax"] + cuisines]

# retrieve svy21 coordinates of restaurants
cv = utils.SVY21()
restaurant_svy21 = [cv.computeSVY21(xy[0], xy[1]) for xy in zip(restaurant_df["lat"], restaurant_df["long"])]
restaurant_df["svy21_x"] = [x[1] for x in restaurant_svy21]
restaurant_df["svy21_y"] = [x[0] for x in restaurant_svy21]

# only filter out Chinese, Indian or Malay
restaurant_df = restaurant_df[(restaurant_df.Chinese == 1) | (restaurant_df.Malay == 1) | (restaurant_df.Indian == 1)]

# Generate Competition Variables

In [4]:
def geodesic_point_buffer(lat, lon, km):
    proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

In [5]:
# create restaurant_points
restaurant_geom = [Point(xy) for xy in zip(restaurant_df["svy21_x"], restaurant_df["svy21_y"])]
restaurant_geom_latlong = [Point(xy) for xy in zip(restaurant_df["lat"], restaurant_df["long"])]
restaurant_geodf = gpd.GeoDataFrame(restaurant_df, geometry=restaurant_geom)
restaurant_geodf["geometry_latlong"] = restaurant_geom_latlong

# retrieve radius around restaurant
radius_05km = []
radius_1km = []
for i, row in restaurant_df.iterrows():
    radius_05km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 0.5)]))
    radius_1km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 1)]))
restaurant_df["radius_05km"] = radius_05km
restaurant_df["radius_1km"] = radius_1km

In [6]:
chi_res_prop_05, mal_res_prop_05, ind_res_prop_05 = [], [], []
chi_res_prop_1, mal_res_prop_1, ind_res_prop_1 = [], [], []

for i, row in restaurant_df.iterrows():
    restaurant_radius_05km = row["radius_05km"]
    filtered_restaurant_05km = restaurant_df[restaurant_df.geometry_latlong.apply(
        lambda x: restaurant_radius_05km.contains(x)
    )]
    filtered_restaurant_05km = filtered_restaurant_05km[filtered_restaurant_05km.name != row["name"]]
    chi_res_prop_05.append(filtered_restaurant_05km.Chinese.mean())
    mal_res_prop_05.append(filtered_restaurant_05km.Malay.mean())
    ind_res_prop_05.append(filtered_restaurant_05km.Indian.mean())
    
    restaurant_radius_1km = row["radius_1km"]
    filtered_restaurant_1km = restaurant_df[restaurant_df.geometry_latlong.apply(
        lambda x: restaurant_radius_1km.contains(x)
    )]
    filtered_restaurant_1km = filtered_restaurant_1km[filtered_restaurant_1km.name != row["name"]]
    chi_res_prop_1.append(filtered_restaurant_1km.Chinese.mean())
    mal_res_prop_1.append(filtered_restaurant_1km.Malay.mean())
    ind_res_prop_1.append(filtered_restaurant_1km.Indian.mean())

restaurant_df["competition_chinese_proportion_05"] = chi_res_prop_05
restaurant_df["competition_malay_proportion_05"] = mal_res_prop_05
restaurant_df["competition_indian_proportion_05"] = ind_res_prop_05

restaurant_df["competition_chinese_proportion_1"] = chi_res_prop_05
restaurant_df["competition_malay_proportion_1"] = mal_res_prop_05
restaurant_df["competition_indian_proportion_1"] = ind_res_prop_05

restaurant_df.to_csv('./data/restaurant_processed.csv', index=False)

# Process Subzone Data

In [7]:
def convert_polygon_svy21_to_latlong(svy21_polygon):
    if type(svy21_polygon) == Polygon:
        N, E = svy21_polygon.exterior.coords.xy
        N, E = [x for x in N], [x for x in E]
        cv = utils.SVY21()
        return Polygon([cv.computeLatLon(xy[0], xy[1]) for xy in zip(N, E)])
    else:
        multipolys = [convert_polygon_svy21_to_latlong(poly) for poly in svy21_polygon]
        return MultiPolygon(multipolys)

In [8]:
# load subzone data
subzone_demographics_df = pd.read_excel("./data/subzone_demographics/subzone_demographics.xlsx")
subzone_geodf = gpd.read_file("./data/maps/master-plan-2014-subzone-boundary-no-sea-shp/MP14_SUBZONE_NO_SEA_PL.shp")

# derive a common key
subzone_demographics_df["subzone_name"] = subzone_demographics_df["Subzone"].apply(lambda x: x.lower())
subzone_geodf["subzone_name"] = subzone_geodf["SUBZONE_N"].apply(lambda x: x.lower())

# combine both dataframes
subzone_combined_df = pd.merge(subzone_demographics_df, subzone_geodf, on="subzone_name")
subzone_combined_df = subzone_combined_df.drop(["subzone_name", "OBJECTID", "SUBZONE_NO"], axis=1)

# retrieve subzone lat long geom
subzone_combined_df["geometry_latlong"] = subzone_combined_df["geometry"].apply(lambda x: convert_polygon_svy21_to_latlong(x))

# filter for relevant subzones
subzone_combined_df = subzone_combined_df.replace("-", 0)
subzone_combined_df = subzone_combined_df[subzone_combined_df.Total != 0]
subzone_combined_df = subzone_combined_df[(subzone_combined_df.Chinese != 0) | (subzone_combined_df.Malays != 0) | (subzone_combined_df.Indians != 0) | (subzone_combined_df.Others != 0)]

# compute proportions
subzone_combined_df["Chinese_Proportion"] = subzone_combined_df["Chinese"] / subzone_combined_df["Total"]
subzone_combined_df["Malay_Proportion"] = subzone_combined_df["Malays"] / subzone_combined_df["Total"]
subzone_combined_df["Indian_Other_Proportion"] = (subzone_combined_df["Indians"] + subzone_combined_df["Others"]) / subzone_combined_df["Total"]

# Option 1: Split Data by Subzones

- A model would look like the following for eatery $i$ in subzone $j$  [each eatery in a unique subzone]: E_ij=alpha + beta X_i  + gamma C_j , where X_i is a vector of share of quota in a radius of 500 meter/1 km [this is specific for eatery i], and C is a vector of controls of the subzone, importantly the proportion of each ethnicity in subzone j. Your betas then measure the effect of the high density of some ethnicity when controlling for the general proportion in an area.
- A second option is that you do it a bit better and take care when an eateries radius goes in two subzones. Then I would for $i$ take the average of the areas in each $j$.

## Retrieve Radius Around Eatery

In [9]:
def geodesic_point_buffer(lat, lon, km):
    proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

In [10]:
restaurant_df = pd.read_csv("./data/restaurant_processed.csv")

# create restaurant_points
restaurant_geom = [Point(xy) for xy in zip(restaurant_df["svy21_x"], restaurant_df["svy21_y"])]
restaurant_geom_latlong = [Point(xy) for xy in zip(restaurant_df["lat"], restaurant_df["long"])]
restaurant_geodf = gpd.GeoDataFrame(restaurant_df, geometry=restaurant_geom)
restaurant_geodf["geometry_latlong"] = restaurant_geom_latlong

In [11]:
# retrieve radius around restaurant
radius_05km = []
radius_1km = []
for i, row in restaurant_df.iterrows():
    radius_05km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 0.5)]))
    radius_1km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 1)]))
restaurant_df["radius_05km"] = radius_05km
restaurant_df["radius_1km"] = radius_1km

## Retrieve Demographic Proportions in Eatery Radius

In [12]:
def compute_intersection_area(radius, polygon):
    try:
        if type(polygon) == Polygon:
            return radius.intersection(polygon).area
        else:
            total_area = [radius.intersection(x).area for x in polygon]
            return sum(total_area)
    except:
        return radius.area # self intersection

In [13]:
chi_prop_05, malay_prop_05, io_prop_05 = [], [], []
chi_prop_1, malay_prop_1, io_prop_1 = [], [], []

for i, row in restaurant_df.iterrows():
    restaurant_radius_05km = row["radius_05km"]
    filtered_subzone_05km = subzone_combined_df[subzone_combined_df.geometry_latlong.apply(lambda x: x.intersects(restaurant_radius_05km))]
    if len(filtered_subzone_05km):
        filtered_area = filtered_subzone_05km.geometry_latlong.apply(
            lambda x: compute_intersection_area(restaurant_radius_05km, x))
        total_area = filtered_area.sum()
        filtered_area_weights = filtered_area / total_area
        chi_prop_05.append((filtered_subzone_05km.Chinese_Proportion * filtered_area_weights).sum())
        malay_prop_05.append((filtered_subzone_05km.Malay_Proportion * filtered_area_weights).sum())
        io_prop_05.append((filtered_subzone_05km.Indian_Other_Proportion * filtered_area_weights).sum())
    else:
        chi_prop_05.append(np.nan)
        malay_prop_05.append(np.nan)
        io_prop_05.append(np.nan)
    
    restaurant_radius_1km = row["radius_1km"]
    filtered_subzone_1km = subzone_combined_df[subzone_combined_df.geometry_latlong.apply(
        lambda x: x.intersects(restaurant_radius_1km))]
    if len(filtered_subzone_05km):
        filtered_area = filtered_subzone_1km.geometry_latlong.apply(
            lambda x: compute_intersection_area(restaurant_radius_1km, x))
        total_area = filtered_area.sum()
        filtered_area_weights = filtered_area / total_area
        chi_prop_1.append((filtered_subzone_1km.Chinese_Proportion * filtered_area_weights).sum())
        malay_prop_1.append((filtered_subzone_1km.Malay_Proportion * filtered_area_weights).sum())
        io_prop_1.append((filtered_subzone_1km.Indian_Other_Proportion * filtered_area_weights).sum())
    else:
        chi_prop_1.append(np.nan)
        malay_prop_1.append(np.nan)
        io_prop_1.append(np.nan)

restaurant_df["chinese_proportion_05"] = chi_prop_05
restaurant_df["malay_proportion_05"] = malay_prop_05
restaurant_df["indian_other_proportion_05"] = io_prop_05

restaurant_df["chinese_proportion_1"] = chi_prop_1
restaurant_df["malay_proportion_1"] = malay_prop_1
restaurant_df["indian_other_proportion_1"] = io_prop_1


TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.2594162033282847 103.8213526190565 at 1.2594162033282847 103.8213526190565
TopologyException: Input geom 1 is

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Inp

TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Input geom 1 is invalid: Ring Self-intersection at or near point 1.3904529671118329 103.94503712167784 at 1.3904529671118329 103.94503712167784
TopologyException: Inp

In [14]:
# select only restaurants with valid proportions 
restaurant_df = restaurant_df.dropna(subset=['chinese_proportion_05', 'malay_proportion_05', 'indian_other_proportion_05'])

## Retrieve HDB Quota Proportion in Eatery Radius

In [15]:
# prepare hdb data
hdb_df = pd.read_csv("./data/hdb_quotas_all.csv")
hdb_geometry = [Point(xy) for xy in zip(hdb_df["latitude"], hdb_df["longitude"])]
hdb_geodf = gpd.GeoDataFrame(hdb_df, geometry=hdb_geometry)

In [16]:
chi_quota_05, malay_quota_05, io_quota_05 = [], [], []
chi_quota_1, malay_quota_1, io_quota_1 = [], [], []

for i, row in restaurant_df.iterrows():
    restaurant_radius_05km = row["radius_05km"]
    filtered_05 = hdb_geodf[hdb_geodf.geometry.apply(lambda x: restaurant_radius_05km.contains(x))]
    filtered_05_perc = filtered_05[["chinese_quota", "malay_quota", "indian_other_quota"]].mean(axis=0)
    chi_quota_05.append(filtered_05_perc["chinese_quota"])
    malay_quota_05.append(filtered_05_perc["malay_quota"])
    io_quota_05.append(filtered_05_perc["indian_other_quota"])
    
    restaurant_radius_1km = row["radius_1km"]
    filtered_1 = hdb_geodf[hdb_geodf.geometry.apply(lambda x: restaurant_radius_1km.contains(x))]
    filtered_1_perc = filtered_1[["chinese_quota", "malay_quota", "indian_other_quota"]].mean(axis=0)
    chi_quota_1.append(filtered_1_perc["chinese_quota"])
    malay_quota_1.append(filtered_1_perc["malay_quota"])
    io_quota_1.append(filtered_1_perc["indian_other_quota"])
    
restaurant_df["chi_quota_05"] = chi_quota_05
restaurant_df["malay_quota_05"] = malay_quota_05
restaurant_df["io_quota_05"] = io_quota_05

restaurant_df["chi_quota_1"] = chi_quota_1
restaurant_df["malay_quota_1"] = malay_quota_1
restaurant_df["io_quota_1"] = io_quota_1

In [17]:
restaurant_df.to_csv("./data/restaurant_processed_option1.csv", index=False)

# Option 2: Compute Proportions Using Subzones

- Make an estimate of the general proportion of each ethnicity in a radius of a eatery. 
- Take a look from an eatery. Lets say there are $rho$ HDB. You know the proportion which have a quota in an ethnicity. You know also how many HDB without a quota are in which subzone. You would take the general estimate of the subzone (if you want adjust it for the number of HDB you know, i.e. you know there are 40% Chinese, but have 7 of 20 HDB where 70% Chinese live, then you adjust the 40%) and calculate the average in those without a quota. Finally you calculate a final variable which weights the one with quota and without quota and takes your overall percentage of each ethnicity in that radius.
- Your model you be just the identity of eatery on the fraction of ethnicities.

## Compute HDB Demographics

In [18]:
# load hdb data
hdb_df = pd.read_csv("./data/hdb_quotas_all.csv")
hdb_geometry = [Point(xy) for xy in zip(hdb_df["svy21_x"], hdb_df["svy21_y"])]
hdb_geodf = gpd.GeoDataFrame(hdb_df, geometry=hdb_geometry)

In [19]:
# load subzone data
subzone_demographics_df = pd.read_excel("./data/subzone_demographics/subzone_demographics.xlsx")
subzone_geodf = gpd.read_file("./data/maps/master-plan-2014-subzone-boundary-no-sea-shp/MP14_SUBZONE_NO_SEA_PL.shp")

# derive a common key
subzone_demographics_df["subzone_name"] = subzone_demographics_df["Subzone"].apply(lambda x: x.lower())
subzone_geodf["subzone_name"] = subzone_geodf["SUBZONE_N"].apply(lambda x: x.lower())

# combine both dataframes
subzone_combined_df = pd.merge(subzone_demographics_df, subzone_geodf, on="subzone_name")
subzone_combined_df = subzone_combined_df.drop(["subzone_name", "OBJECTID", "SUBZONE_NO"], axis=1)

# retrieve subzone lat long geom
subzone_combined_df["geometry_latlong"] = subzone_combined_df["geometry"].apply(lambda x: convert_polygon_svy21_to_latlong(x))

# filter for relevant subzones
subzone_combined_df = subzone_combined_df.replace("-", 0)
subzone_combined_df = subzone_combined_df[subzone_combined_df.Total != 0]
subzone_combined_df = subzone_combined_df[(subzone_combined_df.Chinese != 0) | (subzone_combined_df.Malays != 0) | (subzone_combined_df.Indians != 0) | (subzone_combined_df.Others != 0)]

# compute proportions
subzone_combined_df["Chinese_Proportion"] = subzone_combined_df["Chinese"] / subzone_combined_df["Total"]
subzone_combined_df["Malay_Proportion"] = subzone_combined_df["Malays"] / subzone_combined_df["Total"]
subzone_combined_df["Indian_Other_Proportion"] = (subzone_combined_df["Indians"] + subzone_combined_df["Others"]) / subzone_combined_df["Total"]

### Match Subzone to HDB Block

In [20]:
def retrieve_subzone(point, subzone_df):
    filtered_df = subzone_df[subzone_df.geometry.apply(lambda x: x.contains(point))]
    if len(filtered_df) == 0:
        return None, np.nan, np.nan, np.nan
    subzone_name = filtered_df.iloc[0]["Subzone"]
    subzone_chi_prop = filtered_df.iloc[0]["Chinese_Proportion"]
    subzone_malay_prop = filtered_df.iloc[0]["Malay_Proportion"]
    subzone_io_prop = filtered_df.iloc[0]["Indian_Other_Proportion"]
    return subzone_name, subzone_chi_prop, subzone_malay_prop, subzone_io_prop

# identify subzones for each hdb
hdb_subzone_name, hdb_subzone_chi_prop, hdb_subzone_malay_prop, hdb_subzone_io_prop = [], [], [], []

for i, row in hdb_geodf.iterrows():
    point = row["geometry"]
    s_name, s_c, s_m, s_io = retrieve_subzone(point, subzone_combined_df)
    hdb_subzone_name.append(s_name)
    hdb_subzone_chi_prop.append(s_c)
    hdb_subzone_malay_prop.append(s_m)
    hdb_subzone_io_prop.append(s_io)

hdb_geodf["subzone"] = hdb_subzone_name
hdb_geodf["subzone_chinese_proportion"] = hdb_subzone_chi_prop
hdb_geodf["subzone_malay_proportion"] = hdb_subzone_malay_prop
hdb_geodf["subzone_io_proportion"] = hdb_subzone_io_prop

# drop hdb blocks with no subzone
hdb_geodf = hdb_geodf.dropna(subset=["subzone_chinese_proportion", "subzone_malay_proportion", "subzone_io_proportion"])


### Fill Proportions for Quota Hit Blocks

In [21]:
hdb_geodf["chinese_proportion"] = hdb_geodf["chinese_quota"] * 0.87
hdb_geodf["malay_proportion"] = hdb_geodf["malay_quota"] * 0.25
hdb_geodf["indian_other_proportion"] = hdb_geodf["indian_other_quota"] * 0.15

### Compute Proportions for Non-Quota Hit Blocks

In [22]:
hdb_geodf_proportions_compiled = []
subzone_names = list(set(hdb_geodf["subzone"]))

for subzone in subzone_names:
    hdb_geodf_filtered = hdb_geodf[hdb_geodf.subzone == subzone]

    # compute proportions
    n_total = len(hdb_geodf_filtered)

    subzone_chi_proportion = hdb_geodf_filtered["subzone_chinese_proportion"].iloc[0]
    n_chinese_hit = hdb_geodf_filtered.chinese_quota.sum()
    other_chinese_proportion = ((n_total * subzone_chi_proportion) - (n_chinese_hit * 0.87)) / (n_total - n_chinese_hit)

    subzone_malay_proportion = hdb_geodf_filtered["subzone_malay_proportion"].iloc[0]
    n_malay_hit = hdb_geodf_filtered.malay_quota.sum()
    other_malay_proportion = ((n_total * subzone_malay_proportion) - (n_malay_hit * 0.25)) / (n_total - n_malay_hit)

    subzone_io_proportion = hdb_geodf_filtered["subzone_io_proportion"].iloc[0]
    n_io_hit = hdb_geodf_filtered.indian_other_quota.sum()
    other_io_proportion = ((n_total * subzone_io_proportion) - (n_io_hit * 0.25)) / (n_total - n_io_hit)

    # replace filtered values
    hdb_geodf_filtered["chinese_proportion"] = hdb_geodf_filtered["chinese_proportion"].replace(0, other_chinese_proportion)
    hdb_geodf_filtered["malay_proportion"] = hdb_geodf_filtered["malay_proportion"].replace(0, other_malay_proportion)
    hdb_geodf_filtered["indian_other_proportion"] = hdb_geodf_filtered["indian_other_proportion"].replace(0, other_io_proportion)
    
    hdb_geodf_proportions_compiled.append(hdb_geodf_filtered)


In [23]:
hdb_geodf_processed = pd.concat(hdb_geodf_proportions_compiled, ignore_index=True)
hdb_geodf_processed.to_csv("./data/hdb_quotas_all_processed.csv", index=False)

## Compute Racial Proportions Around Each Eatery

In [24]:
restaurant_df = pd.read_csv("./data/restaurant_processed.csv")

# create restaurant_points
restaurant_geom = [Point(xy) for xy in zip(restaurant_df["svy21_x"], restaurant_df["svy21_y"])]
restaurant_geom_latlong = [Point(xy) for xy in zip(restaurant_df["lat"], restaurant_df["long"])]
restaurant_geodf = gpd.GeoDataFrame(restaurant_df, geometry=restaurant_geom)
restaurant_geodf["geometry_latlong"] = restaurant_geom_latlong

# retrieve radius around restaurant
radius_05km = []
radius_1km = []
for i, row in restaurant_df.iterrows():
    radius_05km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 0.5)]))
    radius_1km.append(Polygon([Point(x[1],x[0]) for x in geodesic_point_buffer(row["lat"], row["long"], 1)]))
restaurant_df["radius_05km"] = radius_05km
restaurant_df["radius_1km"] = radius_1km

In [25]:
# create latitude longitude point
hdb_geodf_processed["geometry_latlong"] = [Point(xy) for xy in zip(
    hdb_geodf_processed["latitude"], hdb_geodf_processed["longitude"])]

In [26]:
res_chi_prop_05, res_malay_prop_05, res_io_prop_05 = [], [], []
res_chi_prop_1, res_malay_prop_1, res_io_prop_1 = [], [], []

for i, row in restaurant_df.iterrows():
    # radius = 0.5km
    restaurant_radius_05 = row["radius_05km"]
    filtered_hdb_df = hdb_geodf_processed[hdb_geodf_processed.geometry_latlong.apply(lambda x: restaurant_radius_05.contains(x))]
    
    # compute proportions
    chi_prop_05 = filtered_hdb_df.chinese_proportion.mean()
    malay_prop_05 = filtered_hdb_df.malay_proportion.mean()
    io_prop_05 = filtered_hdb_df.indian_other_proportion.mean()
    
    res_chi_prop_05.append(chi_prop_05)
    res_malay_prop_05.append(malay_prop_05)
    res_io_prop_05.append(io_prop_05)
    
    # radius = 1km
    restaurant_radius_1 = row["radius_1km"]
    filtered_hdb_df = hdb_geodf_processed[hdb_geodf_processed.geometry_latlong.apply(lambda x: restaurant_radius_1.contains(x))]
    
    # compute proportions
    chi_prop_1 = filtered_hdb_df.chinese_proportion.mean()
    malay_prop_1 = filtered_hdb_df.malay_proportion.mean()
    io_prop_1 = filtered_hdb_df.indian_other_proportion.mean()
    
    res_chi_prop_1.append(chi_prop_1)
    res_malay_prop_1.append(malay_prop_1)
    res_io_prop_1.append(io_prop_1)
    
restaurant_df["chinese_proportion_05"] = res_chi_prop_05
restaurant_df["malay_proportion_05"] = res_malay_prop_05
restaurant_df["indian_other_proportion_05"] = res_io_prop_05

restaurant_df["chinese_proportion_1"] = res_chi_prop_1
restaurant_df["malay_proportion_1"] = res_malay_prop_1
restaurant_df["indian_other_proportion_1"] = res_io_prop_1


In [27]:
# save to dataframe
restaurant_df.to_csv("./data/restaurant_processed_option2.csv", index=False)